In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

from utils import plot_confusion_matrix

2023-05-30 14:12:37.374555: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-30 14:12:39.277025: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 14:12:41.515238: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
rap = xr.open_mfdataset("/glade/u/home/dkimpara/analysis/rap/20221217/0000/*.nc", concat_dim='time', combine='nested')

In [ ]:
# create dewpoint from relative humidity since RAP doesn't natively carry dewpoint
rap['dpt'] = dewpoint_from_relative_humidity(rap['t'] * units.degC,
                                             rap['r'].values / 100)

In [ ]:
rap

In [ ]:
ptypes = ['rain', 'snow', 'icep', 'frzr']

preds = rap.ML_crain + rap.ML_csnow * 2 + rap.ML_cicep * 3 + rap.ML_cfzrn * 4
rap_lbls = rap.crain + rap.csnow * 2 + rap.cicep * 3 + rap.cfrzr * 4

In [ ]:
plot_confusion_matrix()